In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.io as scio
import warnings
import seaborn as sns
from scipy.optimize import root
from matplotlib.font_manager import FontProperties

warnings.filterwarnings('ignore', category=RuntimeWarning)

rural = np.arange(0, 60, 2)
urban = np.arange(1, 60, 2)
east = np.r_[0, 1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 24, 25, 28, 29, 36, 37, 40, 41]
middle = np.r_[6, 7, 22, 23, 26, 27, 30, 31, 32, 33, 34, 35]
west = np.r_[8, 9, 10, 11, 12, 13, 14, 15, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
high_flow_in = np.array([0, 1, 2, 3, 8, 10, 11, 12, 16, 18, 20, 25, 27, 28, 29])
low_flow_in = np.array([5, 24, 23, 14, 9, 6, 26, 7, 4, 22, 21, 13, 19, 15, 17])

font_path = r'C:\Windows\Fonts\simsun.ttc'
chinese_font = FontProperties(fname=r'C:\Windows\Fonts\simsun.ttc')
english_font = FontProperties(fname=r'C:\Windows\Fonts\times.ttf')
pd.options.display.float_format = '{:.4f}'.format

# 定义计算基尼系数的函数
def gini_index(array):
    cum_wealths = np.cumsum(sorted(np.append(array, 0)))
    sum_wealths = cum_wealths[-1]
    xarray = np.array(range(0, len(cum_wealths))) / float(len(cum_wealths)-1)
    yarray = cum_wealths / sum_wealths
    B = np.trapz(yarray, x=xarray)
    A = 0.5 - B
    return A / (A + B)

In [2]:
# 读取基准情形的变量和参数
equilibrium2010 = np.load('./data/equilibrium2010.npz')
R = equilibrium2010['R']
X = equilibrium2010['X']
m = equilibrium2010['m']
pi = equilibrium2010['pi']
L = equilibrium2010['L']
L_bar = equilibrium2010['L_bar']
K = equilibrium2010['K']
w = equilibrium2010['w']
P = equilibrium2010['P']
W = equilibrium2010['W']

parameter = np.load('./data/Parameter.npz')
kappa = parameter['kappa']
theta = parameter['theta']
beta = parameter['beta']
psi_ag = parameter['psi_ag']
psi_na = parameter['psi_na']
alpha_ag = parameter['alpha_ag']
alpha_na = parameter['alpha_na']
sigma_ag = parameter['sigma_ag']
sigma_na = parameter['sigma_na']
eta_ag_ag = parameter['eta_ag_ag']
eta_ag_na = parameter['eta_ag_na']
eta_na_ag = parameter['eta_na_ag']
eta_na_na = parameter['eta_na_na']
phiu = parameter['phiu']
phip = parameter['phip']
chi = parameter['chi']
eta = parameter['eta']
delta = parameter['delta']
xi_ag = parameter['xi_ag']
xi_na = parameter['xi_na']
t = parameter['t']

In [3]:
# 导入外生变量的反事实变化

## 土地供给
K1017 = np.array(pd.read_excel('./data/raw_and_build/城镇村及工矿用地面积.xlsx').iloc[:, 3:])
K1017 = np.delete(K1017, 25, axis=0) # 删除西藏
dK1117 = K1017[:, 1:] / K1017[:, 0].reshape([30, 1])
dK1117 = K1017[:, 1:] / K1017[:, 0].reshape([30, 1])

## 其他外生变量
dKh = np.concatenate([np.ones([30, 1]), dK1117[:, -1].reshape([30, 1])], axis=1).reshape([60, 1])
dKp = np.concatenate([np.ones([30, 1]), dK1117[:, -1].reshape([30, 1])], axis=1).reshape([60, 1])
dT = np.ones([60, 1])
dtau = np.ones([60, 60])
dmu = np.ones([60, 60])

In [4]:
# 求解土地改革下的反事实均衡

def SolveCounter(XX, dK):
    dw = XX[:60].reshape([60, 1])
    dL = XX[60:120].reshape([60, 1])
    dp = XX[120:].reshape([60, 1])
    dKh, dKp = dK, dK

    ## (a)价格指数的变化
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))

    ## (b)公共支出的变化
    dG = dw * dL / dP

    ## (c)单位投入束成本的相对变化
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p

    ## (d)贸易份额的相对变化
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) # 分子(考虑生产函数中存在规模经济效应)
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) # 分母
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi # 反事实情形下的贸易份额

    ## (e)最终品价格的相对变化
    dp1 = dpi_den ** (-1 / theta)

    ## (f)收入倍率的相对变化
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1]) # 计算总工资收入
    w = wL / L # 名义工资收入
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1])  # 计算出城市部门和农村部门的土地总收入
    rho_prime = rKrK * dw * dL / L_bar # 土地改革后，户籍居民的人均土地收入
    rho_prime[1::2] = 0 # 城市部门居民没有土地收入，所以将偶数行替换为0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T # 改革后的收入倍率
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) # 计算delta的主对角线元素
    ddelta = delta_prime # ddelta和delta_prime的非主对角线元素是一样的
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) # 计算并替换ddelta的主对角线元素

    ## (g)流动份额的相对变化
    dg = dG / (dL ** chi) # 人均公共支出的相对变化
    dW = dw / dP # 实际工资的相对变化
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa # 分子
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) # 分母
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm

    ## (h)常住人口数的相对变化
    dL1 = (m_prime @ L_bar) / L

    ## (i)计算反事实下的总产出与总支出
    R_prime = R * dw * dL
    wL_prime = wL * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime ## 根据总产出R_prime计算出的总支出X_prime
    vL_prime = (wL_prime.reshape([30, 2]) * np.array([(alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1]).reshape([1, 2])).reshape([60, 1])
    PG_prime = (wL_prime.reshape([30, 2]) * np.array([t * (alpha_ag + sigma_ag) / (beta + t - beta * t) / alpha_ag, 1 - beta + sigma_na / alpha_na + beta * t]).reshape([1, 2])).reshape([60, 1])
    A_prime = (beta * (1 - t) * np.repeat(vL_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([psi_ag, psi_na]).reshape([1, 2])).reshape([60, 1])
    C_prime = (np.repeat(PG_prime.reshape([30, 2]).sum(axis=1).reshape([30, 1]), 2, axis=1) * np.array([xi_ag, xi_na]).reshape([1, 2])).reshape([60, 1])
    B_ag_prime = ((R_prime.reshape([30, 2]) * np.array([eta_ag_ag, eta_ag_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_na_prime = ((R_prime.reshape([30, 2]) * np.array([eta_na_ag, eta_na_na]).reshape([1, 2])).sum(axis=1)).reshape([30, 1])
    B_prime = np.concatenate([B_ag_prime, B_na_prime], axis=1).reshape([60, 1])
    X_prime1 = A_prime + B_prime + C_prime ## 根据各项支出计算出的总支出X_prime1，均衡下应该与前面的X_prime相等

    ## (j)均衡条件
    res1 = dp1 - dp
    res2 = dL1 - dL
    res3 = X_prime1 - X_prime

    return np.concatenate([res1, res2, res3]).flatten().astype(float)

In [5]:
# 定义计算土地改革下反事实变量的函数

def counterfautual_result(dw, dL, dp, dK):
    dKh, dKp = dK, dK
    dpdp_wp_c = np.repeat((dp.reshape([30, 2]) ** np.array([psi_ag * beta, psi_na * beta]).reshape([1, 2])).prod(axis=1).reshape([30, 1]), 2, axis=1).reshape([60, 1])
    drh = dw * dL / dKh
    dP =  dpdp_wp_c *  (drh ** (1 - beta))
    dG = dw * dL / dP
    dw_wp = (dw.reshape([30, 2]) ** np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1])
    drp = dw * dL / dKp
    drp_wp = (drp.reshape([30, 2]) ** np.array([sigma_ag, sigma_na]).reshape([1, 2])).reshape([60, 1])
    dpdp_wp_p_ag = (dp.reshape([30, 2]) ** np.array([eta_ag_ag, eta_na_ag]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p_na = (dp.reshape([30, 2]) ** np.array([eta_ag_na, eta_na_na]).reshape([1, 2])).prod(axis=1).reshape([30, 1])
    dpdp_wp_p = np.concatenate([dpdp_wp_p_ag, dpdp_wp_p_na], axis=1).reshape([60, 1])
    dc = dw_wp * drp_wp * dpdp_wp_p
    dpi_num = dT * ((dtau * dc) ** (-theta)) * (dG ** (phip * theta)) * (dL ** (theta * delta)) # 分子(考虑生产函数中存在规模经济效应)
    dpi_den = (dpi_num * pi).sum(axis=0).reshape([60, 1]) # 分母
    dpi = (dpi_num.T / dpi_den).T
    pi_prime = pi * dpi # 反事实情形下的贸易份额
    wL = (R.reshape([30, 2]) * np.array([alpha_ag, alpha_na]).reshape([1, 2])).reshape([60, 1]) # 计算总工资收入
    w = wL / L # 名义工资收入
    w_prime = w * dw
    coef_rK_ag = (alpha_ag + sigma_ag - (beta + t - beta * t) * alpha_ag) / ((beta + t - beta * t) * alpha_ag)
    coef_rK_na = sigma_na / alpha_na + (1 - beta) * (1 - t)
    rKrK = (wL.reshape([30, 2]) * np.array([coef_rK_ag, coef_rK_na]).reshape([1, 2])).reshape([60, 1])  # 计算出城市部门和农村部门的土地总收入
    rho_prime = rKrK * dw * dL / L_bar # 土地改革后，户籍居民的人均土地收入
    rho_prime[1::2] = 0 # 城市部门居民没有土地收入，所以将偶数行替换为0
    delta_prime = 1 + (w_prime ** (-1)) @ rho_prime.T # 改革后的收入倍率
    delta_diag = ((L / (L_bar * np.diag(m).reshape([60, 1]))).reshape([30, 2]) * np.array([coef_rK_ag, 0]).reshape([1, 2]) + 1).reshape([60, 1]) # 计算delta的主对角线元素
    ddelta = delta_prime # ddelta和delta_prime的非主对角线元素是一样的
    np.fill_diagonal(ddelta, np.diag(delta_prime).reshape([60, 1]) / delta_diag) # 计算并替换ddelta的主对角线元素
    dg = dG / (dL ** chi) # 人均公共支出的相对变化
    dW = dw / dP # 实际工资的相对变化
    dm_num = (ddelta * (dmu ** (-1)) * (dg ** phiu) * dW) ** kappa # 分子
    dm_den = (dm_num * m).sum(axis=0).reshape([60, 1]) # 分母
    dm = (dm_num.T / dm_den).T
    m_prime = m * dm
    R_prime = R * dw * dL
    X_prime = np.linalg.inv(pi_prime) @ R_prime ## 根据总产出R_prime计算出的总支出X_prime

    return dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag

In [6]:
# 定义生成一次模拟结果的函数

def simulation_program():
    
    ## 随机生成新增建设用地的分配方式
    num_prov = 30
    random_numbers = np.random.rand(num_prov)
    sum_random_numbers = np.sum(random_numbers)
    land_per_prov = (dK_total2017 * random_numbers / sum_random_numbers)
    dK = np.concatenate([np.ones([30, 1]), ((land_per_prov + K2010) / K2010).reshape([30, 1])], axis=1).reshape([60, 1])
    #dK = (K1017[:, -1] / K1017[:, 0]).reshape([31, 1]) ## 现实情形，用于测试程序

    ## 计算该分配模式下全国人均实际GDP、居民福利以及省际人均GDP基尼系数
    XX = root(SolveCounter, x0=np.ones(3*60), args=(dK,)) # 求解非线性方程组
    dw = XX.x[:60].reshape([60, 1])
    dL = XX.x[60:120].reshape([60, 1])
    dp = XX.x[120:].reshape([60, 1])
    dg, dc, dpi, pi_prime, dP, ddelta, dW, dm, m_prime, R_prime, X_prime, drh, dG, delta_diag = counterfautual_result(dw, dL, dp, dK)
    W = w / P
    w_prime = w * dw
    L_prime = L * dL
    P_prime = P * dP
    W_prime = w_prime / P_prime
    dW_country = (((W_prime * L_prime).sum() / L_prime.sum()) / ((W * L).sum() / L.sum()) - 1) * 100
    dincgini_country = (gini_index(W_prime) / gini_index(W) - 1) * 100
    
    coef_g_na = 1 - beta + sigma_na / alpha_na + beta * t
    coef_g_ag = (alpha_ag + sigma_ag) * t / (beta + t - beta * t) / alpha_ag
    g = ((w * L).reshape([30, 2]) * np.array([coef_g_ag, coef_g_na]).reshape([1, 2])).reshape([60, 1]) / P / (L ** chi)
    dV = dW * ((dg) ** phiu) * (np.diag(dm).reshape([-1, 1]) ** (-1 / kappa)) * (np.diag(ddelta).reshape([-1, 1]))
    weight = (np.diag(m).reshape([-1, 1]) ** (-1 / kappa)) * (g ** phiu) * W * L_bar * delta_diag
    weight_country = weight / weight.sum()
    dV_country = ((dV * weight_country).sum() - 1) * 100
    
    return dK, dW_country, dincgini_country, dV_country
    

In [7]:
# 生成50万次模拟结果(1)：准备工作，生成一个500000行的dataframe

#columns_names = ['dK' + str(i) for i in range(60)] + ['dW_country', 'dincgini_country', 'dV_country']
#simulation_results = pd.DataFrame(np.nan, index=range(500000), columns=columns_names)
#simulation_results.to_csv('./data/Simulation_results.csv', index=False)


In [8]:
# 生成100万次模拟结果(2)：计算2017年相比2010年新增的建设用地面积

K1017 = np.array(pd.read_excel('./data/raw_and_build/城镇村及工矿用地面积.xlsx').iloc[:, 3:])
K1017 = np.delete(K1017, 25, axis=0) # 删除西藏
K2010 = K1017[:, 0]
dK_total2017 = (K1017[:, -1] - K1017[:, 0]).sum()

In [18]:
# 生成100万次模拟结果(3)：逐次模拟，由于运算时间长，加入了允许中断的设计

simulation_results = pd.read_csv('./data/Simulation_results.csv')
for index, row in simulation_results.iterrows():
    if np.isnan(simulation_results.iloc[index, 0])==True:
        dK, dW_country, dincgini_country, dV_country = simulation_program()
        each_results = np.concatenate(
            [dK.flatten(), np.array(dW_country).flatten(), np.array(dincgini_country).flatten(), np.array(dV_country).flatten()]
        )
        simulation_results.iloc[index, :] = each_results
        print('Finished: ' + str(index+1) + '/500000', end='\r')
        if index % 1000 == 1:
            simulation_results.to_csv('./data/Simulation_results.csv', index=False)

simulation_results.to_csv('./data/Simulation_results.csv', index=False)
print('Finished: ' + str(index+1) + '/500000', end='\r')
print('\nFinished!')

Finished: 489809/500000
Finished!
